In [223]:
# !pip install requests
# !pip install beautifulsoup4
# !pip install wikipedia
# !pip install PyDictionary
# !pip install pytrends
# !pip install urbandictionary

In [224]:
import pytrends
from pytrends.request import TrendReq
import wikipedia
import requests
from bs4 import BeautifulSoup
import random
import PyDictionary
from PyDictionary import PyDictionary
#import spacy
import pandas as pd
import re

dir(pytrends)
pytrends = TrendReq(hl='en-US', tz=360) # create a pytrends object
#pytrends.trending_searches(pn='united_states') # trending searches in real time for United States
# create a dictionary of the top 1000 trending searches for this year in the United States (as of 2022)
trending_searches_dict = pytrends.top_charts(
    date='2022',    # specifies the time period as the last year
    geo='US',            # specifies the country as the United States
)
real_time_trending_searches_dict = pytrends.trending_searches(pn='united_states') # trending searches in real time for United States



# show how many trending searches there are
print(f'Retrieved {len(trending_searches_dict)} trending searches.')
# show the top 10 trending searches in real time for United States
trending_searches_dict.head(10)

Retrieved 10 trending searches.


,title,exploreQuery
0,Wordle,
1,Election results,
2,Betty White,
3,Queen Elizabeth,
4,Bob Saget,
5,Ukraine,
6,Mega Millions,
7,Powerball numbers,
8,Anne Heche,
9,Jeffrey Dahmer,


In [225]:
import urbandictionary as ud
import re
# adding import alias recommended
bad_patterns = [r'sex*', r'porn*',r'fuck*',r'-ass*','ass','shit',r'damn*',r'ass|asse*',r'cock*',r'whor*',r'nigg*',r'slut*','blowjob',r'fagg*',r'boob|boob*', r'bitch*', r'bastard*',' ho |hoe',r'breast*|jugs', r'cunt*', r'puss*', r'dick*', 'naked', r'nud*', r'masterb*',r'mastu',r'nipple*',r'penis|penal|peni*','god','jesus','christ','bible','church','religion','pray','prayer','faith','lord','allah','muslim','islam','allah','ejaculate','jew*','islamic','atheist',r'rapist*|rape*',r'pedo*','atheism','atheists','atheist','atheists','christian','christianity','christians','christian','christians','gay',r'tit*|titt*','god','jesus','christ','bible','church','religion','pray','prayer','faith','lord','allah','muslim','islam','allah','islamic','atheist','atheism','atheists','atheist','atheists','christian','christianity','christians','christian','christians','yahwey','yeshua',r'israel*',r'sex*', r'porn*',r'fuck*',r'-ass*','ass','shit',r'damn*',r'ass|asse*',r'cock*',r'whor*',r'nigg*',r'slut*','blowjob',r'fagg*',r'boob|boob*', r'breast*|jugs', r'cunt*', r'puss*', r'dick*', 'naked', r'nud*', r'nipple*',r'penis|penal|peni*','god','jesus','christ','bible','church','religion','pray','prayer','faith','lord','allah','muslim','islam','allah','islamic','atheist','atheism','atheists','atheist','atheists','christian','christianity','christians','christian','christians','gay',r'tit*|titt*', 'fellatio', 'fuck', 'nigger','lynch',r'erotic*']

# to make the game more fun we can replace words with less inflamatory ones.


def check_for_badwords(definition, bad_patterns):
    definition = definition.lower()
    # if any of the buzzwords are found return true else false
    if any(re.search(r'\b' + word + r'\b', definition) for word in bad_patterns):
        return True
    else:
        return False

def check_for_good_patterns(definition, title):
    # check both title and definition for good patterns
    good_patterns = [r'\b\w+phobia\.?\b', r'\bslang\b', r'\bacronymn\b', r'\bmeme\b']


    if any(re.match(r'\b' + word + r'\b', definition) for word in good_patterns):
        print(f'Found a good pattern in the definition: {definition}')
        return True
    elif any(re.match(r'\b' + word + r'\b', title) for word in good_patterns):
        print(f'Found a good pattern in the title: {title}')
        return True
    else:
        return False

def remove_undesireable_sentences(definition, title, bad_patterns):
    # convert the definition and title to lowercase
    definition = definition.lower()
    title = title.lower()

    # split the definition into sentences
    sentences = definition.split('.')

    # split the title into words
    title_words = title.split()

    # initialize a list to store the acceptable sentences
    acceptable_sentences = []

    # iterate over the sentences in the definition
    for sentence in sentences:
        # split the sentence into words
        sentence_words = sentence.split()

        # check if any of the words in the sentence are also in the title
        if not any(word in title_words for word in sentence_words):
            # if not, add the sentence to the list of acceptable sentences
            acceptable_sentences.append(sentence)

    # remove sentences that are not in English
    definition = re.sub(r'[^\x00-\x7f]',r'', '.'.join(acceptable_sentences))
    # remove sentences that contain a regex match to any word in the buzzwords list
    definition = re.sub(r'|'.join(map(re.escape, bad_patterns)), '', definition)
    return definition


def unpack_definitions(definition):
    # remove the brackets and clean up the definitions
    # with regex
    definition = definition.replace("[","")
    definition = definition.replace("]","")
    definition = definition.replace("'","")
    definition = definition.replace('"',"")
    definition = definition.replace("(","")
    definition = definition.replace(")","")

    # remove double spaces
    definition = definition.replace('  ',' ')

    return definition

import wikipedia

def get_page_length(phrase):
  # search for pages on Wikipedia that match the given phrase
    try:
        pages = wikipedia.search(phrase)

        # retrieve the first page from the search results
        page = wikipedia.page(pages[0])
        print(f'Found the page for {page.title}', end='')
        if phrase.lower() != page.title.lower():
            print(' ... nevermind... not the right page.')
            return 0
        # return the length of the page
        return len(page.content)
    except Exception as e:
            return 0


In [232]:

# example: usage example,
# upvotes: number of upvotes on Urban Dictionary,
# downvotes: number of downvotes on Urban Dictionary
import time
wikitest = False # set to true to test the wikipedia page length
# include a phrase if it has a combined total of at least 100 upvotes and downvotes on Urban Dictionary
rand_dict = {}
total_votes_thresh = 200 # min number of upvotes + downvotes
upvotes_thresh = 100 # min number of upvotes
downvotes_thresh = 10 # max number of downvotes
desired_number_of_cards = 15
rands = [] # the randoms

while len(rand_dict) < desired_number_of_cards:
  time.sleep(1)
  rand = ud.random() # returns a list of 5 random phrases and definitions from Urban Dictionary
  # append these to a master list
  rands.extend(rand) # rands is a list of all the random phrases and definitions from Urban Dictionary

  # iterate over the elements in the rand object
  for element in rand:
    # extract the relevant data from the element
    phrase = element.word
    definition = element.definition
    usage_example = element.example
    upvotes = element.upvotes
    downvotes = element.downvotes
    # define a list of boolean values
    values = [upvotes + downvotes >= total_votes_thresh,
              upvotes >= upvotes_thresh and downvotes <= downvotes_thresh,
              check_for_good_patterns(definition, phrase)]
    

    # check if any element in the list is True
    if any(values) and not check_for_badwords(definition, bad_patterns) \
      and not check_for_badwords(phrase, bad_patterns): # check if the phrase or definition contains any buzzwords
    # include the element in the dictionary if it has a combined total of at least 100 upvotes and downvotes on Urban Dictionary
      definition = remove_undesireable_sentences(
        definition, phrase, bad_patterns) # remove sentences that contain buzzwords, are not in English, etc.
      definition = unpack_definitions(definition) # remove brackets and clean up the definitions with regex
      if wikitest:
        print('Testing phrase:', phrase, ' for popularity on Wikipedia...')
        page_length = get_page_length(phrase) # get the length of the Wikipedia page for the phrase
        print('Page length:', page_length)
        if page_length < 1000: # if the page is too short, skip it
          continue
      
      print(f'Added {phrase}: {definition[0:10]}...')
      rand_dict[phrase] = definition

# save the cards to a csv file
import pandas as pd
df = pd.DataFrame.from_dict(rand_dict, orient='index')
df.to_csv('cards.csv', header=False)

# save the cards to a json file
import json
with open('cards.json', 'w') as f:
    json.dump(rand_dict, f)
  
# Hippopotomonstrosesquippedaliophobia - fear of long words

Added aint no thang but a chicken wang: doesnt mat...
Added akansha: a really c...
Added Todd: a man that...
Added Sexing: the act of...
Added hit the bricks: 1.a someti...
Added Mason: used to de...
Added fb: fb is an a...
Added soul patch: n. the bea...
Added fanfiction.net: a stupid s...
Added Bumvertising: when a mer...
Added woo: a cigarett...
Found a good pattern in the definition: slang for [baby mama], [wifey], [girlfriend]
Added bussy baby: slang for ...
Added Linsanity: the energy...
Added Kate: girl that ...
Added ash hole: when a bow...
Added plush: hot, aweso...


In [227]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def has_sexual_comment(phrase, bad_patterns):
    # analyze the sentiment of the phrase
    scores = analyzer.polarity_scores(phrase)
    # check if the compound score is greater than or equal to 0.5, which indicates a positive sentiment
    if scores['compound'] <= 0.5:
        print(f' * Found a negative sentiment in the phrase: {phrase}')
        print(f'Score: {scores}')
    
        # if the phrase also contains a buzzword, return True
        if any(re.match(pattern, phrase) for pattern in bad_patterns):
            print(f' ** Also, Found a bad pattern in the phrase: {phrase}')
            return True
        else:
            return False
    else:
        return False


In [228]:
test_phrase = "a single or collection of females that you shamefully solicit for sexual acts but what be disparaged if anyone knew that you were hooking up with them."

has_sex = has_sexual_comment(test_phrase, bad_patterns)

print('Testing for sex in the phrase, {}: {}'.format(test_phrase,has_sex))

 * Found a negative sentiment in the phrase: a single or collection of females that you shamefully solicit for sexual acts but what be disparaged if anyone knew that you were hooking up with them.
Score: {'neg': 0.174, 'neu': 0.826, 'pos': 0.0, 'compound': -0.6187}
Testing for sex in the phrase, a single or collection of females that you shamefully solicit for sexual acts but what be disparaged if anyone knew that you were hooking up with them.: False


In [229]:
test_phrase =  "another phrased coined by david alan weinshel wheeler 02 bu 06. simplifies the stress of saying how and come separately. also eliminates the need to hit hte spacebar when typing."
has_sex = has_sexual_comment(test_phrase, bad_patterns)

print('Testing for sex in the phrase, {}: {}'.format(test_phrase,has_sex))

 * Found a negative sentiment in the phrase: another phrased coined by david alan weinshel wheeler 02 bu 06. simplifies the stress of saying how and come separately. also eliminates the need to hit hte spacebar when typing.
Score: {'neg': 0.088, 'neu': 0.912, 'pos': 0.0, 'compound': -0.4215}
Testing for sex in the phrase, another phrased coined by david alan weinshel wheeler 02 bu 06. simplifies the stress of saying how and come separately. also eliminates the need to hit hte spacebar when typing.: False


In [230]:
real_time_trending_searches_dict

,0
0,Earthquake
1,Mat Ishbia
2,The Specials
3,Weather Chicago
4,Boston Bruins
5,Green Bay Packers
6,Jalen Hurts
7,Giants
8,New York Giants
9,Trump


In [231]:
import requests
import random
import re
from bs4 import BeautifulSoup
from datetime import datetime
from PyDictionary import PyDictionary
import wikipedia

dictionary = PyDictionary() # instantiate 

def get_wiki_definition(card):
  # Get the summary of the Wikipedia page
  summary = wikipedia.summary(card)
  return summary

# # Example usage:
# card = "John Smith"
# definition = get_definition(card)
# print(definition)  # prints the summary of the Wikipedia page "John Smith"

def get_definition(card):
  # Check the type of the card
  if is_slang_phrase(card):
    # Get the definition of the slang phrase from Urban Dictionary
    url = "https://api.urbandictionary.com/v0/define?term=" + card
    response = requests.get(url)
    data = response.json()
    definition = data['list'][0]['definition']
  elif is_person(card):
    # Get the summary of the person from Wikipedia
    definition = get_wiki_definition(card[0]) # for a card title get the definition (summary) of the wikipedia page.
  else:
    # this is a word so use PyDictionary library.
    definition = dictionary.meaning(str(card[0])) # this could have mult. meanings returned. 
    definition = definition[0] # the first meaning. 

def get_random_slang_phrase():
  # Make a request to the Urban Dictionary API to get a random slang phrase
  url = "https://api.urbandictionary.com/v0/random"
  response = requests.get(url)
  data = response.json()
  slang_phrase = data['list'][0]['word']
  return slang_phrase

def get_random_wiki_person():
  # Make a request to Wikipedia API to get a random person
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  person = data['query']['random'][0]['title']
  return person

def get_random_wiki_word():
  # Make a request to Wikipedia API to get a random word
  url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=random&rnnamespace=0&rnlimit=1"
  response = requests.get(url)
  data = response.json()
  word = data['query']['random'][0]['title']
  return word

def create_a_card_deck(num_cards=30):
  # create 30 random cards with equal distributions of slang, people, and words.
  # avoid profanity, or lude sexual topics.
  
  # Create a list of all the cards
  cards = []

  # Initialize a dictionary to keep track of the count of each type of card
  counts = {
    "slang": 0,
    "people": 0,
    "words": 0
  }

  # While there are fewer than 30 cards in the deck, generate a new card
  while len(cards) < num_cards:
    # Find the type of card with the lowest count
    min_count_type = min(counts, key=counts.get)

    # Generate a new card of that type
    if min_count_type == "slang":
      slang_phrase = get_random_slang_phrase()
      new_card = (slang_phrase, "slang", get_definition(slang_phrase))
    elif min_count_type == "people":
      person = get_random_wiki_person()
      new_card = (person, "people", get_definition(person))
    else:
      word = get_random_wiki_word()
      new_card = (word, "word", get_definition(word))

    # Add the new card to the list and update the count
    cards.append(new_card)
    counts[min_count_type] += 1 
  return cards

def is_slang_phrase(card):
  if card[1] == 'slang':
    return True
  else:
    return False

def is_person(card):
  if card[1] == 'person':
    return True
  else:
    return False

def filter_cards(cards):
  # create a function to filter out inappropriate content
  filtered_cards = []
  for card in cards:

    # check if the card contains profanity or sexual content
    print("checking card: {}".format(card))
    definition = card[2] # definition
    card_type = card[1] # i.e. 'slang'
    word = card[0] # word 
    try:
      if "profanity" in definition or "sex" in definition or "lewd" in definition:
        continue
      elif check_for_badwords():
        continue
      else:
        filtered_cards.append(card)
    except Exception as e:
      print(f'error with finding the meaning in the soup object. soup comes from the urban dictionary page.')
      print(e)
  return filtered_cards

# generate a deck of cards
card_deck = create_a_card_deck()

# filter the deck for inappropriate content
filtered_deck = filter_cards(card_deck)

# print the cards
print('Generated {} cards'.format(len(filtered_deck)))

KeyError: 0

In [ ]:
# print the filtered cards
for card in filtered_deck:
  print(card)


In [ ]:
# Example usage:
card = "dope"
definition = get_definition(card)
card_type = "phrase"
card_preview = 

# (slang_phrase, "slang", get_definition(slang_phrase))
    


print(definition)  # prints the definition of the slang phrase "dope"

card = "John Smith"
definition = get_definition(card)
print(definition)  # prints the summary of the person "John Smith"

card = "tree"
definition = get_definition(card)
print(definition)  # prints the definition of the word "tree"
